In [2]:
"eee upgrading".find('upgrad')

4

In [3]:
'upgrad' in "eee upgrading"

True

In [1]:
import pandas as pd
import re
import itertools
import numpy as np
from uti import DataLoader, Logger
from Broker import ricsregion
from Eikon import Eikon_update_price_enhanced
from Crawler import REPORT_TYPE_DICT

DL = DataLoader()
logger = Logger()

Current root directory: C:/Users/JayYang/OneDrive - Alpha Sherpa Capital
Current Base directory: C:/Users/JayYang/OneDrive - Alpha Sherpa Capital/Citi
Current Database directory: C:/Users/JayYang/OneDrive - Alpha Sherpa Capital/Citi/Database


In [2]:
from Model.settings import DataCleaner

Engine current settings: 
daily_position 18 
region_position 6 
skewness: 0
Engine current settings: 
daily_position 18 
region_position 6 
skewness: 0


In [3]:
from Database.rating_change import tpc_scanner, rc_scanner

Current working directory C:\Users\JayYang\Platform\Citi


In [11]:
DC = DataCleaner()

In [19]:
X_train = DL.loadDB("Backtest/Benchmark data/train.csv", parse_dates=['Time'])
X_test = DL.loadDB("Backtest/Benchmark data/test.csv", parse_dates=['Time'])
tpc_df = pd.concat([X_train, X_test], axis=0)

16:49:29 Loading C:/Users/JayYang/OneDrive - Alpha Sherpa Capital/Citi/Database\Backtest/Benchmark data/train.csv
16:49:33 Loading C:/Users/JayYang/OneDrive - Alpha Sherpa Capital/Citi/Database\Backtest/Benchmark data/test.csv


In [25]:
tpc_df.columns

Index(['Date', 'Ticker', 'Tickers', 'company', 'Headline', 'Region',
       'Industry', 'Head analyst', 'Report Type', 'Report Types', 'pubId',
       'Headline sentiment', 'Summary sentiment', 'Sector', 'atr', 'atrx',
       'atr_used', 'gap', 'market_cap_usd', 'volume_d_10_sma', 'prev1_date',
       'prev1_open', 'prev1_high', 'prev1_low', 'prev1_close', 'd0_date',
       'd0_open', 'd0_high', 'd0_low', 'd0_close', 'd1_date', 'd1_open',
       'd1_high', 'd1_low', 'd1_close', 'd2_date', 'd2_open', 'd2_high',
       'd2_low', 'd2_close', 'ticker', 'exch_location', 'exch_region',
       'date_local', 'time_local', 'release_period', 'ticker_updated',
       'gap_in_atr', 'TPS_new', 'TPS', 'TPS_prev', 'deltaTPS', 'deltaTPS%',
       'abs(deltaTPS)', 'RC_upgrade', 'RC_downgrade', 'pm_region', 'delta_r',
       'exch_region2', 'market_cap_grp', 'side', 'stop_loss', 'd0_r', 'd1_r',
       'd2_r'],
      dtype='object')

In [21]:
senti_df = DL.loadDB('Citi sentiment.csv', parse_dates=['Time'])

16:49:36 Loading C:/Users/JayYang/OneDrive - Alpha Sherpa Capital/Citi/Database\Citi sentiment.csv


In [22]:
senti_df = senti_df.set_index(['Time', 'Summary'])
tpc_df = tpc_df.set_index(['Time', 'Summary'])

In [26]:
col_to_add = ['TPS_new', 'TPS', 'TPS_prev', 'deltaTPS', 'deltaTPS%', 'abs(deltaTPS)', 'RC_upgrade', 'RC_downgrade']
senti_df.loc[tpc_df.index, col_to_add] = tpc_df[col_to_add]

In [31]:
senti_df_new = senti_df[~senti_df.index.isin(tpc_df.index)]
senti_df_old = senti_df[senti_df.index.isin(tpc_df.index)]

,,Date,Ticker,Tickers,company,Headline,Region,Industry,Head analyst,Report Type,Report Types,...,Headline sentiment,Summary sentiment,TPS_new,TPS,TPS_prev,deltaTPS,deltaTPS%,abs(deltaTPS),RC_upgrade,RC_downgrade
Time,Summary,,,,,,,,,,,,,,,,,,,,,
2022-05-04 13:15:00,"We update our model to factor in Baosteel’s 2021&1Q22 results and our latest steel supply-demand forecast. We also introduce 2024E earnings estimates and lower our TP to Rmb9/sh (from Rmb11.2), based on 1.0x 2022E BVPS, to reflect our lowered GP/t assumptions. Maintain Buy.",2022-05-04 00:00:00,600019.SS,600019.SS,Baoshan Iron & Steel,Baoshan Iron &amp; Steel (600019.SS): Model Up...,Asia,Steel,"Jack Shang, CFA",Target Price Change,"Target Price Change,Estimate Change",...,negative,negative,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-05-04 11:02:00,"4Q earnings missed - the trading update already pointed to a disruptive quarter (retail sales dipped -3% YoY) as Omicron impacted Jan and trends in March were also subdued (gold price volatility; weak sentiment); adjusted PAT (excluding VRS spends) came in at Rs5.4bn (+2% YoY), below Citi/consensus estimates of Rs6.5bn/Rs6.6bn. There are unusual/one-off items (ex-gratia payments, associate meet expenses) that have weighed on reported earnings, adjusting for which the miss would’ve been lower. Moreover, the company continued to invest behind longer-term initiatives. Looking ahead, outlook for FY23E seems positive – management alluded to good April offtake as gold prices somewhat settled. Titan’s fundamental drivers remain intact, benefitting from both top-down tailwinds for organized jewellers and its bottom-up initiatives that should drive incremental share gains. We’ll await details from next week’s analyst meet. Maintain Buy. We make non-material changes to FY23/24 forecasts (<1%) post results; New TP of Rs2,890 (vs. Rs2,910).",2022-05-04 00:00:00,TITN.BO,TITN.BO,Titan Company,"Titan Company (TITN.BO): Unexciting 4QFY22, Bu...",Asia,Household Appliances,Aditya Mathur,Earnings Review,"Target Price Change,Estimate Change,Earnings R...",...,positive,negative,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-05-04 11:00:00,"In this report we resume coverage on A2A after a long period of restriction with a Buy rating and a target price of €2.50, implying a 61% ETR. Amid the adverse impact of the draught in Italy on the group short term results, we believe the market has overlooked the material improvement of the company outlook fueled by the pressure to reduce Italy gas dependency that could translate into material opportunities for the company in its waste and district heating division. As power prices should support 2023 Earnings, A2A trades below 10X PE, and offers an attractive 6% DY. Solid balance sheet and structural growth also underpin our recommendation.",2022-05-04 00:00:00,A2.MI,"A2.MI,ENEI.MI,TRN.MI,IG.MI,SRG.MI",A2A,A2A (A2.MI): The overlooked Energy transition ...,Western Europe,Diversified Utilities,Antonella Bianchessi,Rating Change,"Target Price Change,Estimate Change,Rating Change",...,positive,positive,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-05-04 02:14:00,"MMFS reported net profit of Rs6bn (4x yoy, -33% qoq), ahead of our estimate as lower provisions (significant decline in stress + deferment of net NPA target) offset lower operating profit (reversal of income following RBI direction + higher opex for collections). GS3 has fallen 130bps yoy after rising 650bps in 1Q due to the second wave. MMFS is targeting 2x AUM by FY25 and reduced volatility in earnings by improved customer targeting - success on these can be a significant positive. Senior appointments made over the past year include CRO / COO / tech head from leading banks. NNPA must be <6% by Oct ’22 as per RBI norms (current level unavailable) – MMFS does not expect this to need abnormal provisions. We change FY23 / 24 PAT by 0% / +9% (build higher growth offset by higher provisions in FY23 – recoveries front-loaded in FY22); target Rs220 (from Rs190; SOTP).",2022-05-04

In [39]:
DL.toDB(senti_df_new.reset_index(), 'senti_df_new.csv')
DL.toDB(senti_df_old.reset_index(), 'senti_df_old.csv')

In [49]:
def tpc_scanner2(summary_list: list) -> list:
    """
        Scan target price from the summaries of the reports that are classified as Target Price Change.

    """
    tpc_list = []
    r = re.compile(r'\btarget price [a-z]{0,3}[ ]{0,1}[\$|\€\£\¥]{0,1}(\d*\.?\d+)(?<!%)|'
                   r'\bprice target [a-z]{0,3}[ ]{0,1}[\$|\€\£\¥]{0,1}(\d*\.?\d+)(?<!%)|'

                   r'\bpt of [a-z]{0,3}[ ]{0,1}[\$|\€\£\¥]{0,1}(\d*\.?\d+)(?<!%)|'
                   r'\btp of [a-z]{0,3}[ ]{0,1}[\$|\€\£\¥]{0,1}(\d*\.?\d+)(?<!%)|'
                   r'\btarget price of [a-z]{0,3}[ ]{0,1}[\$|\€\£\¥]{0,1}(\d*\.?\d+)(?<!%)|'
                   r'\bprice target of [a-z]{0,3}[ ]{0,1}[\$|\€\£\¥]{0,1}(\d*\.?\d+)(?<!%)|'
                   r'\bpt to [a-z]{0,3}[ ]{0,1}[\$|\€\£\¥]{0,1}(\d*\.?\d+)(?<!%)|'
                   r'\btp to [a-z]{0,3}[ ]{0,1}[\$|\€\£\¥]{0,1}(\d*\.?\d+)(?<!%)|'
                   r'\btarget price to [a-z]{0,3}[ ]{0,1}[\$|\€\£\¥]{0,1}(\d*\.?\d+)(?<!%)|'
                   r'\bprice target to [a-z]{0,3}[ ]{0,1}[\$|\€\£\¥]{0,1}(\d*\.?\d+)(?<!%)|'

                   r'\bpt at [a-z]{0,3}[ ]{0,1}[\$|\€\£\¥]{0,1}(\d*\.?\d+)(?<!%)|'
                   r'\btp at [a-z]{0,3}[ ]{0,1}[\$|\€\£\¥]{0,1}(\d*\.?\d+)(?<!%)|'
                   r'\btarget price at [a-z]{0,3}[ ]{0,1}[\$|\€\£\¥]{0,1}(\d*\.?\d+)(?<!%)|'
                   r'\bprice target at [a-z]{0,3}[ ]{0,1}[\$|\€\£\¥]{0,1}(\d*\.?\d+)(?<!%)|'

                   r'\bpt is now set at [a-z]{0,3}[ ]{0,1}[\$|\€\£\¥]{0,1}(\d*\.?\d+)(?<!%)|'
                   r'\btp is now set at [a-z]{0,3}[ ]{0,1}[\$|\€\£\¥]{0,1}(\d*\.?\d+)(?<!%)|'
                   r'\btarget price is now set at [a-z]{0,3}[ ]{0,1}[\$|\€\£\¥]{0,1}(\d*\.?\d+)(?<!%)|'
                   r'\bprice target is now set at [a-z]{0,3}[ ]{0,1}[\$|\€\£\¥]{0,1}(\d*\.?\d+)(?<!%)|'

                   r'\bpt rises to [a-z]{0,3}[ ]{0,1}[\$|\€\£\¥]{0,1}(\d*\.?\d+)(?<!%)|'
                   r'\btp rises to [a-z]{0,3}[ ]{0,1}[\$|\€\£\¥]{0,1}(\d*\.?\d+)(?<!%)|'
                   r'\btarget rises to [a-z]{0,3}[ ]{0,1}[\$|\€\£\¥]{0,1}(\d*\.?\d+)(?<!%)|'
                   r'\bprice rises to [a-z]{0,3}[ ]{0,1}[\$|\€\£\¥]{0,1}(\d*\.?\d+)(?<!%)|'

                   r'\bpt now at [a-z]{0,3}[ ]{0,1}[\$|\€\£\¥]{0,1}(\d*\.?\d+)(?<!%)|'
                   r'\btp now at [a-z]{0,3}[ ]{0,1}[\$|\€\£\¥]{0,1}(\d*\.?\d+)(?<!%)|'
                   r'\btarget price now at [a-z]{0,3}[ ]{0,1}[\$|\€\£\¥]{0,1}(\d*\.?\d+)(?<!%)|'
                   r'\bprice target now at [a-z]{0,3}[ ]{0,1}[\$|\€\£\¥]{0,1}(\d*\.?\d+)(?<!%)|'
                   r'\bpt is [a-z]{0,3}[ ]{0,1}[\$|\€\£\¥]{0,1}(\d*\.?\d+)(?<!%)|'
                   r'\btp is [a-z]{0,3}[ ]{0,1}[\$|\€\£\¥]{0,1}(\d*\.?\d+)(?<!%)|'
                   r'\btarget price is [a-z]{0,3}[ ]{0,1}[\$|\€\£\¥]{0,1}(\d*\.?\d+)(?<!%)|'
                   r'\bprice target is [a-z]{0,3}[ ]{0,1}[\$|\€\£\¥]{0,1}(\d*\.?\d+)(?<!%)|'
                   r'\bpt are [a-z]{0,3}[ ]{0,1}[\$|\€\£\¥]{0,1}(\d*\.?\d+)(?<!%)|'
                   r'\btp are [a-z]{0,3}[ ]{0,1}[\$|\€\£\¥]{0,1}(\d*\.?\d+)(?<!%)|'
                   r'\btarget price are [a-z]{0,3}[ ]{0,1}[\$|\€\£\¥]{0,1}(\d*\.?\d+)(?<!%)|'
                   r'\bprice target are [a-z]{0,3}[ ]{0,1}[\$|\€\£\¥]{0,1}(\d*\.?\d+)(?<!%)|'
                   r'\bwith a [a-z]{0,3}[ ]{0,1}[\$|\€\£\¥]{0,1}(\d*\.?\d+)(?<!%) tp|'
                   r'\bwith a [a-z]{0,3}[ ]{0,1}[\$|\€\£\¥]{0,1}(\d*\.?\d+)(?<!%) pt|'
                   r'\bwith a [a-z]{0,3}[ ]{0,1}[\$|\€\£\¥]{0,1}(\d*\.?\d+)(?<!%) target price|'
                   r'\bwith a [a-z]{0,3}[ ]{0,1}[\$|\€\£\¥]{0,1}(\d*\.?\d+)(?<!%) price target|'

                   r'\b[a-z]{0,3}[ ]{0,1}[\$|\€\£\¥]{0,1}(\d*\.?\d+)(?<!%) tp|'
                   r'\b[a-z]{0,3}[ ]{0,1}[\$|\€\£\¥]{0,1}(\d*\.?\d+)(?<!%) pt|'
                   r'\b[a-z]{0,3}[ ]{0,1}[\$|\€\£\¥]{0,1}(\d*\.?\d+)(?<!%) target price|'
                   r'\b[a-z]{0,3}[ ]{0,1}[\$|\€\£\¥]{0,1}(\d*\.?\d+)(?<!%) price target|'

                   r'\bprice target revised [a-z]*? to? [a-z]{0,3}[ ]{0,1}[\$|\€\£\¥]{0,1}(\d*\.?\d+)(?<!%)|'
                   r'\btarget price .*?to? [a-z]{0,3}[ ]{0,1}[\$|\€\£\¥]{0,1}(\d*\.?\d+)(?<!%)|'
                   r'\bprice target .*?to? [a-z]{0,3}[ ]{0,1}[\$|\€\£\¥]{0,1}(\d*\.?\d+)(?<!%)|'
                   r'\bpt .*?to? [a-z]{0,3}[ ]{0,1}[\$|\€\£\¥]{0,1}(\d*\.?\d+)(?<!%)|'
                   r'\btp .*?to? [a-z]{0,3}[ ]{0,1}[\$|\€\£\¥]{0,1}(\d*\.?\d+)(?<!%)|'

                   r'\bpt [a-z]*? from.*? to? [a-z]{0,3}[ ]{0,1}[\$|\€\£\¥]{0,1}(\d*\.?\d+)(?<!%)|'
                   r'\btp [a-z]*? from.*? to? [a-z]{0,3}[ ]{0,1}[\$|\€\£\¥]{0,1}(\d*\.?\d+)(?<!%)|'
                   r'\btarget price [a-z]*? from.*? to? [a-z]{0,3}[ ]{0,1}[\$|\€\£\¥]{0,1}(\d*\.?\d+)(?<!%)|'
                   r'\bprice target [a-z]*? from.*? to? [a-z]{0,3}[ ]{0,1}[\$|\€\£\¥]{0,1}(\d*\.?\d+)(?<!%)|'

                   r'\bpt revised [a-z]*? to? [a-z]{0,3}[ ]{0,1}[\$|\€\£\¥]{0,1}(\d*\.?\d+)(?<!%)|'
                   r'\btp revised [a-z]*? to? [a-z]{0,3}[ ]{0,1}[\$|\€\£\¥]{0,1}(\d*\.?\d+)(?<!%)|'
                   r'\btarget price revised [a-z]*? to? [a-z]{0,3}[ ]{0,1}[\$|\€\£\¥]{0,1}(\d*\.?\d+)(?<!%)|'

                   , flags=re.I)

    count = 0
    years_tbd = map(str, range(2019, 2031))

    for summary in summary_list:
        summary = str(summary).replace(',', '')

        result = r.findall(summary)
        result = [x for x in list(itertools.chain(*result)) if len(x) > 0]
        result = [x for x in result if x not in years_tbd]
        if len(result) > 0:
#             print(summary)
#             print(result)
            if len(np.unique(result)) > 1:
                # Store all the possible target prices for ARTIFICIAL intelligence to select
                tpc_list.append(str(result))
            elif result[0] is not None:
                tpc_list.append(str(result[0]))
            else:
                tpc_list.append(None)
        else:
            tpc_list.append(None)

    logger.info(f'Total number of {count} of TPC needs to modify with the help of ARTIFICIAL intelligence.')

    return tpc_list

In [84]:
senti_df_new.loc[:, ['TPS_new', 'RC_upgrade', 'RC_downgrade']] = ''
TPC_mask = senti_df_new['Report Types'].fillna('').str.contains('Target Price Change')
RC_mask = senti_df_new['Report Types'].fillna('').str.contains('Rating Change')

In [86]:
tpc_list = tpc_scanner2(senti_df_new[TPC_mask].reset_index()['Summary'])
senti_df_new.loc[TPC_mask, 'TPS_new'] = tpc_list

rc_list = rc_scanner(senti_df_new[RC_mask].reset_index()['Summary'])
senti_df_new.loc[RC_mask, 'RC_upgrade'] = np.array(rc_list)[:, 0]
senti_df_new.loc[RC_mask, 'RC_downgrade'] = np.array(rc_list)[:, 1]

18:31:09 Total number of 0 of TPC needs to modify with the help of ARTIFICIAL intelligence.


In [90]:
senti_df_new.loc[TPC_mask, 'TPS_new'].reset_index()

,Time,Summary,TPS_new
0,2022-05-04 13:15:00,We update our model to factor in Baosteel’s 20...,9
1,2022-05-04 11:02:00,4Q earnings missed - the trading update alread...,2890
2,2022-05-04 11:00:00,In this report we resume coverage on A2A after...,2.50
3,2022-05-04 02:14:00,"MMFS reported net profit of Rs6bn (4x yoy, -33...",None
4,2022-05-04 00:36:00,"Suning’s 4Q21 results came below expectation, ...",None
...,...,...,...
1282,2017-08-29 11:00:00,We have updated our ARM and Assore numbers as ...,"['252', '115']"
1283,2017-06-01 11:00:00,Looking over an earnings cliff - Earnings mome...,None
1284,2017-03-31 11:00:00,Reporting season recap - The SA diversified mi...,None
1285,2017-03-10 22:55:00,"Downgrade from Buy to Neutral, TP R7.65 - RPC ...",7.65


In [55]:
def rc_scanner(summary_list: list) -> list:
    """
        Scan rating change from the summaries of the reports that are classified as Rating Change.

    """
    rc = []

    for summary in summary_list:
        summary = str(summary).replace(',', '')

        upgrade_result = "upgrad" in summary.lower()
        downgrade_result = "downgrad" in summary.lower()

        rc.append([upgrade_result, downgrade_result])

    return rc

array([False, False,  True, ...,  True, False,  True])

In [91]:
DL.toDB(senti_df_new.reset_index(), 'senti_df_new.csv')